# Implement K-Means algorithm with Tensorflow, and apply it to classify handwriten digit images.

## Importing the libraries

In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

## Ignore all GPUs, tf random forest does not benefit from it.

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Importing MNIST data

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Setting the Parameters

## Total steps to train

In [6]:
num_steps = 50 

## The number of samples per batch

In [7]:
batch_size = 1024

## The number of clusters

In [8]:
k = 25

## The 10 digits

In [9]:
num_classes = 10

## Each image is 28x28 pixels

In [10]:
num_features = 784

## Input images

In [11]:
X = tf.placeholder(tf.float32, shape=[None, num_features])

## Labels

In [12]:
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

## K-means parameter

In [15]:
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine', use_mini_batch=True)

## Build KMeans graph

In [20]:
training_graph = kmeans.training_graph()
 
if len(training_graph) > 6: # Tensorflow 1.4+
    (all_scores, cluster_idx, scores, cluster_centers_initialized,
     cluster_centers_var, init_op, train_op) = training_graph
else:
    (all_scores, cluster_idx, scores, cluster_centers_initialized,
     init_op, train_op) = training_graph

## Fix for cluster_idx being a tuple

In [21]:
cluster_idx = cluster_idx[0]
avg_distance = tf.reduce_mean(scores)

## Initialize the variable

In [22]:
init_vars = tf.global_variables_initializer()

## Start TensorFlow session

In [23]:
sess = tf.Session()

## Run the initializer

In [24]:
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

True

## Training

In [25]:
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                        feed_dict={X: full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
Step 20, Avg Distance: 0.220328
Step 30, Avg Distance: 0.219776
Step 40, Avg Distance: 0.219419
Step 50, Avg Distance: 0.219154


## Assign a label to each centroid
## Count total number of labels per centroid, using the label of each training
## sample to their closest centroid (given by 'idx')

In [26]:
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]

## Assign the most frequent label to the centroid

In [27]:
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

## Evaluation ops
## Lookup: centroid_id -> label

In [28]:
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)

## Compute accuracy

In [29]:
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Test Model

In [30]:
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.7127
